<a href="https://colab.research.google.com/github/AeroEng16/FootballAnalysis/blob/main/inferenceTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import re
import json
import datetime
import pandas as pd
from PIL import Image, ImageDraw
from io import BytesIO
import yaml
import os
try:
    from roboflow import Roboflow
except ImportError as e:
    !pip install roboflow

try:
  from ultralytics import YOLO
except ImportError as e:
  !pip install ultralytics
  from ultralytics import YOLO

;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
robowFlowAPIKey ="e4YWipTNVce3CfUmX7fc"

In [ ]:
imageURL = "https://raw.githubusercontent.com/AeroEng16/FootballAnalysis/main/oldFirmTestImage.png?raw=true"
imageURL

response = requests.get(imageURL)
img = Image.open(BytesIO(response.content))
img1 = ImageDraw.Draw(img)


#display(img)

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO("yolov8n.pt")

# Define path to the image file
source = "path/to/image.jpg"

# Run inference on the source
results = model(source)  # list of Results objects